In [ ]:
# Author: Naveen Lalwani
# Script to distill knowledge of LeNet-5 model trained on MNIST to student model
import keras
import numpy as np
import tensorflow as tf

config = tf.ConfigProto()
config.gpu_options.allow_growth = True

from keras.datasets import mnist
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Flatten, Activation
from keras.models import Sequential, Model
from keras import layers
from tensorflow.keras import losses
from tensorflow.keras.utils import to_categorical

Using TensorFlow backend.


**Preprocessing the MNIST dataset for LeNet5**

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Storing for smaller model
x_train_small = x_train
x_test_small = x_test

# Reshaping for LeNet 5
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

# Normalization
x_train = x_train / 255
x_test = x_test / 255

# One hot encoding
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

11493376/11490434 [==============================] - 1s 0us/step


# **Defining LeNet 5 Model**

In [ ]:
model = Sequential()

model.add(Conv2D(6, kernel_size=(5, 5), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(16, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(120, activation='relu'))

model.add(Dense(84, activation='relu'))

# Defining activation function separately here to get the logits/ dark knowledge from teacher model
model.add(Dense(10, name="logits"))
model.add(Activation('softmax'))          

# Run model
# Adam Optimization
# Loss = Cross Entropy Loss
# Epochs = 20
model.compile(loss=keras.metrics.categorical_crossentropy, optimizer=keras.optimizers.Adam(), metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=128, epochs= 20, verbose=1, validation_data=(x_test, y_test))

# Get Test Accuracy
score = model.evaluate(x_test, y_test)
print('Test Loss:', score[0])
print('Test accuracy:', score[1])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 38s 641us/step - loss: 0.3330 - acc: 0.9006 - val_loss: 0.1047 - val_acc: 0.9668
Epoch 2/20
60000/60000 [==============================] - 38s 634us/step - loss: 0.0939 - acc: 0.9703 - val_loss: 0.0621 - val_acc: 0.9790
Epoch 3/20
60000/60000 [==============================] - 37s 621us/step - loss: 0.0661 - acc: 0.9791 - val_loss: 0.0506 - val_acc: 0.9831
Epoch 4/20
60000/60000 [==============================] - 37s 622us/step - loss: 0.0529 - acc: 0.9831 - val_loss: 0.0548 - val_acc: 0.9826
Epoch 5/20
60000/60000 [==============================] - 37s 617us/step - loss: 0.0436 - acc: 0.9858 - val_loss: 0.0384 - val_acc: 0.9885
Epoch 6/20
60000/60000 [==============================] - 37s 624us/step - loss: 0.0376 - acc: 0.9880 - val_loss: 0.0395 - val_acc: 0.9

In [ ]:
getSoftmaxKnowledge = Model(inputs=model.input, outputs=model.get_layer("logits").output)
model_logits = getSoftmaxKnowledge.predict(x_train)

In [ ]:
# Defining function described by Geoffrey Hinton in his paper of Knowledge Distillation
def softmax_with_temperature(logits, temperature):
    logits = logits / temperature
    return (np.exp(logits) / np.sum(np.exp(logits)))

In [ ]:
# Temperature is a hyperparameter
temperature = 7
softened_train_prob = softmax_with_temperature(model_logits, temperature)

# **Defining Student Model**

In [ ]:
# Model Definition for the Student Model
def build_small_model():
    inputs = layers.Input(shape = (784,))
    
    x = layers.Dense(50, activation='relu', name='FC1')(inputs)
    
    x = layers.Dense(10, name='logits')(x)
    
    preds = layers.Activation('softmax', name='Softmax')(x)
  
    model = Model(inputs=inputs, outputs=preds)
    model.summary()
    return model

In [ ]:
small_model = build_small_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 784)               0         
_________________________________________________________________
FC1 (Dense)                  (None, 50)                39250     
_________________________________________________________________
logits (Dense)               (None, 10)                510       
_________________________________________________________________
Softmax (Activation)         (None, 10)                0         
Total params: 39,760
Trainable params: 39,760
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Preprocessing for smaller model
x_train = x_train_small.reshape(-1, 784)
x_test = x_test_small.reshape(-1, 784)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255
    
    
# Optimization = N Adam
# Loss = Cross Entropy loss
# Epochs = 50
# Trained with dark knowledge
small_model.compile(optimizer='Nadam', loss= 'categorical_crossentropy', metrics=['categorical_accuracy'])
small_model.fit(x_train, softened_train_prob, epochs=50, batch_size=128)

test_loss, test_acc = small_model.evaluate(x_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)

Epoch 1/50
60000/60000 [==============================] - 2s 31us/step - loss: 1.6473e-05 - categorical_accuracy: 0.8496
Epoch 2/50
60000/60000 [==============================] - 2s 27us/step - loss: 1.4338e-05 - categorical_accuracy: 0.9258
Epoch 3/50
60000/60000 [==============================] - 2s 26us/step - loss: 1.3950e-05 - categorical_accuracy: 0.9404
Epoch 4/50
60000/60000 [==============================] - 2s 26us/step - loss: 1.3760e-05 - categorical_accuracy: 0.9487
Epoch 5/50
60000/60000 [==============================] - 2s 26us/step - loss: 1.3645e-05 - categorical_accuracy: 0.9536
Epoch 6/50
60000/60000 [==============================] - 2s 26us/step - loss: 1.3567e-05 - categorical_accuracy: 0.9571
Epoch 7/50
60000/60000 [==============================] - 2s 27us/step - loss: 1.3509e-05 - categorical_accuracy: 0.9603
Epoch 8/50
60000/60000 [==============================] - 2s 26us/step - loss: 1.3464e-05 - categorical_accuracy: 0.9621
Epoch 9/50
60000/60000 [========

In [ ]:
small_model.save('model_50.h5')
model.save('model_LeNet5.h5')